# Hugging Face Accelerate: Treinando modelos em diferentes ambientes

Uma loja virtual está buscando soluções para melhorar os serviços prestados e entender os pontos positivos e negativos dos produtos disponíveis na plataforma de vendas.

Atualmente os clientes podem enviar análises de produtos comprados, porém não há uma maneira de analisar esses dados de forma eficiente.

Nosso papel nesse projeto é criar um modelo de IA capaz de fazer a classificação dessas análises, para que seja possível entender a satisfação dos clientes e motivos das análises.

## Carregando os dados

Para carregar os dados, vamos usar o `load_dataset` da biblioteca 🤗 Datasets. A partir dela será feito o download e o **cache** da base de dados (dessa maneira o download não precisa ser feito se reiniciar o notebook).

Vamos utilizar a base de dados `multilingual-NLI`, contendo dados de premissas e hipóteses em diversas linguagens para a criação de modelos de inferência de linguagem:

- https://huggingface.co/datasets/MoritzLaurer/multilingual-NLI-26lang-2mil7

In [ ]:
# !pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
url = 'https://huggingface.co/datasets/MoritzLaurer/multilingual-NLI-26lang-2mil7/resolve/main/data/pt_anli-00000-of-00001-21ca95ba4e8fa69b.parquet'

In [ ]:
dados = load_dataset('parquet', data_files = url)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dados

DatasetDict({
    train: Dataset({
        features: ['premise_original', 'hypothesis_original', 'label', 'premise', 'hypothesis'],
        num_rows: 25000
    })
})

In [ ]:
dados['train'][0]

{'premise_original': 'Based on the evidence of the health threats of tobacco, based on the thousands of deaths from tobacco related diseases and the terrible toll that tobacco addiction takes on individuals in society, it is very difficult to imagine any reason for not supporting the bill and its important amendments to the Tobacco Act.',
 'hypothesis_original': 'The health threats of tobacco are not known',
 'label': 2,
 'premise': 'Com base na evidência das ameaças à saúde do tabaco, com base nos milhares de mortes por doenças relacionadas ao tabaco e no terrível preço que a dependência do tabaco afeta os indivíduos na sociedade, é muito difícil imaginar qualquer motivo para não apoiar o projeto de lei e suas importantes alterações à Lei do Tabaco.',
 'hypothesis': 'As ameaças à saúde do tabaco não são conhecidas'}

In [ ]:
dados['train'].to_pandas()

,premise_original,hypothesis_original,label,premise,hypothesis
0,Based on the evidence of the health threats of...,The health threats of tobacco are not known,2,Com base na evidência das ameaças à saúde do t...,As ameaças à saúde do tabaco não são conhecidas
1,"Harry Brand (October 20, 1895 – February 22, 1...",Harry Brand lived in America.,0,"Descrito como ""o mentor que fez Shirley Temple...",Harry Brand viveu na América.
2,The jet ski accident<br>Wendy was just an amat...,wendy drove too fast while on a jet ski,0,O acidente de jet skibr>Wendy era apenas um mo...,wendy dirigiu muito rápido enquanto em um jet ski
3,"This is the episode list of ""Mr. Bean"", a Brit...",Mr. Bean was both a live action and an animate...,1,"Esta é a lista de episódios de ""Mr. Bean"", uma...",Mr. Bean foi ao mesmo tempo uma ação ao vivo e...
4,Leading the movement which has always been mor...,the agent is aged 27,1,Liderando o movimento que sempre esteve mais e...,o agente tem 27 anos
...,...,...,...,...,...
24995,The Journal of the Evangelical Theological Soc...,The Evangelical Theological Society is read by...,1,O Jornal da Sociedade Teológica Evangélica é u...,A Sociedade Teológica Evangélica é lida pelos ...
24996,Waiting on A Laptop<br>Kate bought a laptop on...,the laptop was supposed to arrive at her moms,0,Esperando em um laptopbr>Kate comprou um lapto...,O laptop deveria chegar a suas mães
24997,Mr Carstens left his job as IMF's deputy manag...,Mr Calderon is not IMF's deputy managing direc...,0,Carstens deixou seu emprego como vice-diretor-...,Calderon não é o vice-diretor-gerente do FMI.
24998,RSC Anderlecht Féminin is a Belgian women's fo...,A belgian sport team for girls had been founde...,0,RSC Anderlecht Féminin é uma equipa belga de f...,Uma equipe esportiva belga para meninas havia ...


## Separando dados de treino e teste

Para fazer o treinamento do modelo de forma apropriada, vamos selecionar apenas os dados em português e separar os dados em treino e teste, para que seja possível realizar a avaliação de desempenho.

In [ ]:
dados_selecionados = dados.select_columns(['premise', 'hypothesis', 'label'])
dados_selecionados['train'].to_pandas()

,premise,hypothesis,label
0,Com base na evidência das ameaças à saúde do t...,As ameaças à saúde do tabaco não são conhecidas,2
1,"Descrito como ""o mentor que fez Shirley Temple...",Harry Brand viveu na América.,0
2,O acidente de jet skibr>Wendy era apenas um mo...,wendy dirigiu muito rápido enquanto em um jet ski,0
3,"Esta é a lista de episódios de ""Mr. Bean"", uma...",Mr. Bean foi ao mesmo tempo uma ação ao vivo e...,1
4,Liderando o movimento que sempre esteve mais e...,o agente tem 27 anos,1
...,...,...,...
24995,O Jornal da Sociedade Teológica Evangélica é u...,A Sociedade Teológica Evangélica é lida pelos ...,1
24996,Esperando em um laptopbr>Kate comprou um lapto...,O laptop deveria chegar a suas mães,0
24997,Carstens deixou seu emprego como vice-diretor-...,Calderon não é o vice-diretor-gerente do FMI.,0
24998,RSC Anderlecht Féminin é uma equipa belga de f...,Uma equipe esportiva belga para meninas havia ...,0


In [ ]:
dados_selecionados['train'].train_test_split(test_size = 0.2, shuffle = True)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5000
    })
})

In [ ]:
dados_selecionados = dados_selecionados['train'].train_test_split(test_size = 0.2, shuffle = True)
dados_selecionados

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 5000
    })
})

In [ ]:
dados_selecionados['train'].to_pandas()

,premise,hypothesis,label
0,"Matsuzaka ( , Matsuzaka Daisuke, ; nascido em ...",Daisuke Matsuzaka jogou por várias equipes da ...,2
1,Duke é um personagem fictício da série de brin...,Duke é um personagem imaginário.,0
2,Os AIR Charts são os gráficos oficiais de vend...,"Apresentado pela AIR, a Australian Independent...",1
3,San Francisco Bay Ferry é um serviço de ferry ...,As crianças devem ser supervisionadas enquanto...,1
4,Enquanto os países bálticos estão prestes a ad...,Esses países têm um rápido aumento no sucesso ...,0
...,...,...,...
19995,"O que é certo, no entanto, é que cerca de 120....",Os Homo Sapiens existem há um milhão de anos.,2
19996,Mohanned Mehdi Al-Nadawi (em árabe: ) um fu...,Hamid Reza Ebrahimi não destacou Al-Nadawi em ...,0
19997,Johnny Reno é um filme americano de 1966 feito...,Johnny Reno foi libertado quando os baby boome...,0
19998,Como cultivar gaurabr>Semear sementes dentro d...,"Ao fazer seus planos para suas plantas gaura, ...",0


In [ ]:
dados_selecionados['test'].to_pandas()

,premise,hypothesis,label
0,The Stranger Left No Card (1952) é um curta-me...,The Stranger Left No Card foi um filme que foi...,0
1,"Mas quando você entra em uma sala de aula, voc...",O narrador está insatisfeito com as escolas de...,0
2,Oleg Smirnov (nascido em 8 de abril de 1980) é...,Smirnov foi redigido em 1996.,2
3,Um dia com Wilbur Robinson é um livro infantil...,William Joyce morreu em 2007.,1
4,Backyard Campingbr>John e seus amigos estavam ...,O pai do John é assustador.,1
...,...,...,...
4995,Daoud Abdel Sayed (em árabe: ] ) é um diret...,Sayed nunca ganhou nenhum prêmio internacional,2
4996,O Fórum de Conservação de Territórios Ultramar...,A UKOTCF não se baseia na Noruega.,0
4997,"No domingo 14 de dezembro de 2008, um jornalis...",Muntazer al-Zaidi gosta de fazer trens modelo.,1
4998,O Construtor Universal de John von Neumann é u...,O Construtor Universal de John von Neumann cau...,1


## Preparando os dados

Antes de alimentar o nosso modelo com os dados, precisamos prepará-los. Isso pode ser feito com o `Tokenizer` da biblioteca Transformers. Ele faz a tokenização dos inputs, convertendo para IDs com base no vocabulário pré-treinado. Esse é o formato que o modelo de linguagem espera receber.

Para isso, vamos instanciar um tokenizador com o `AutoTokenizer.from_pretrained` que garatne que a tokenização feita corresponde à arquitetura que vamos utilizar do modelo.

Vamos utilizar um modelo BERT pré-treinado disponível no Hugging Face:

- https://huggingface.co/Geotrend/bert-base-pt-cased

In [ ]:
# !pip install -q transformers

In [ ]:
checkpoint_modelo = 'Geotrend/bert-base-pt-cased'

In [ ]:
from transformers import AutoTokenizer

tokenizador = AutoTokenizer.from_pretrained(checkpoint_modelo)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/162k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizador(dados_selecionados['train']['premise'][0])

{'input_ids': [11, 21384, 14749, 427, 21, 25, 21384, 14749, 427, 15288, 10919, 25, 40, 18083, 348, 333, 203, 4019, 203, 527, 22, 144, 368, 294, 10543, 203, 509, 407, 5627, 11054, 22422, 310, 91, 19796, 17378, 947, 7993, 14170, 27, 3395, 24984, 310, 91, 2173, 1167, 14964, 25, 365, 501, 4593, 206, 81, 5150, 1637, 206, 240, 2759, 833, 7054, 21, 13559, 22, 81, 91, 15724, 1458, 8881, 240, 15336, 7396, 7054, 21, 17063, 947, 22, 27, 12], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def funcao_tokenizadora(linha):
    return tokenizador(
        linha['premise'],
        linha['hypothesis'],
        truncation = True,
        padding = 'max_length',
        max_length = 128,
        return_tensors = 'pt'
    )

In [ ]:
funcao_tokenizadora(dados_selecionados['train'][:5])

{'input_ids': tensor([[   11, 21384, 14749,   427,    21,    25, 21384, 14749,   427, 15288,
         10919,    25,    40, 18083,   348,   333,   203,  4019,   203,   527,
            22,   144,   368,   294, 10543,   203,   509,   407,  5627, 11054,
         22422,   310,    91, 19796, 17378,   947,  7993, 14170,    27,  3395,
         24984,   310,    91,  2173,  1167, 14964,    25,   365,   501,  4593,
           206,    81,  5150,  1637,   206,   240,  2759,   833,  7054,    21,
         13559,    22,    81,    91, 15724,  1458,  8881,   240, 15336,  7396,
          7054,    21, 17063,   947,    22,    27,    12, 15288, 10919, 21384,
         14749,   427, 24984,   280,  6512, 13544,   240,  4177,  2484,    12,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
print(tokenizador.sep_token)

[SEP]


In [ ]:
print(tokenizador.sep_token_id)

12


In [ ]:
dados_tokenizados = dados_selecionados.map(funcao_tokenizadora, batched = True, remove_columns = ['premise', 'hypothesis'])
dados_tokenizados

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})

In [ ]:
dados_tokenizados['train'].to_pandas()

,label,input_ids,token_type_ids,attention_mask
0,2,"[11, 21384, 14749, 427, 21, 25, 21384, 14749, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,"[11, 3381, 144, 368, 9520, 2354, 2573, 14434, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,1,"[11, 1412, 16508, 946, 6371, 1559, 463, 22229,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[11, 494, 1252, 1928, 12088, 144, 368, 12070, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,0,"[11, 16151, 463, 2925, 19191, 12359, 2779, 607...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
19995,2,"[11, 60, 220, 144, 8698, 25, 286, 6708, 25, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
19996,0,"[11, 9463, 1238, 3582, 1132, 16216, 734, 26, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
19997,0,"[11, 2987, 12742, 144, 368, 2742, 3594, 203, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
19998,0,"[11, 2426, 15567, 14608, 1301, 1066, 10013, 43...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
dados_tokenizados['test'].to_pandas()

,label,input_ids,token_type_ids,attention_mask
0,0,"[11, 216, 14645, 9578, 599, 10364, 21, 848, 22...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,"[11, 3351, 1352, 18110, 7383, 348, 469, 4853, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2,"[11, 17410, 11402, 21703, 799, 21, 18083, 348,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,"[11, 2567, 608, 303, 14276, 18381, 532, 2279, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1,"[11, 3148, 7862, 4689, 318, 10013, 43, 459, 81...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...
4995,2,"[11, 1262, 13700, 23922, 6274, 402, 21, 348, 8...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4996,0,"[11, 60, 51, 5454, 490, 203, 1780, 1733, 421, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4997,1,"[11, 599, 18846, 331, 203, 5505, 203, 296, 25,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4998,1,"[11, 60, 1780, 11551, 1259, 2958, 203, 459, 26...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Preparando dados da variável alvo

Assim com os dados textuais, a variável alvo também precisa ser transformada para um formato adequado para utilização no modelo. Vamos realizar a transformação para o formato ClassLabel e ajustar o nome da coluna para 'labels', que é o nome esperado pelo modelo.

In [ ]:
dados_tokenizados['train'].features

{'label': Value(dtype='int64', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
from datasets import ClassLabel


A label de classificação possui os seguintes valores:

- `entailment` = 0
- `neutral` = 1
- `contradiction`= 2

In [ ]:
classes = ClassLabel(num_classes =3 , names = ['entailment', 'neutral', 'contradiction'])
classes

ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None)

In [ ]:
dados_tokenizados = dados_tokenizados.cast_column('label', classes)

Casting the dataset:   0%|          | 0/20000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
dados_tokenizados = dados_tokenizados.rename_column('label', 'labels')

In [ ]:
dados_tokenizados.set_format('torch')

In [ ]:
dados_tokenizados['train'].features

{'labels': ClassLabel(names=['entailment', 'neutral', 'contradiction'], id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

## Criando função de dataloaders

Antes de treinar o modelo, precisamos dividir os dados em lotes. Isso é importante porque a maioria dos modelos de deep learning não processa todo o conjunto de dados de uma vez, mas sim em pequenos lotes, o que ajuda a reduzir o uso de memória e a tornar o treinamento mais eficiente.

In [ ]:
# !pip install -q torch

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def criar_dataloaders(batch_size_treino = 8, batch_size_validacao = 32):
    dataloader_treino = DataLoader(dados_tokenizados['train'], shuffle = True, batch_size = batch_size_treino)
    dataloader_validacao = DataLoader(dados_tokenizados['test'], shuffle = False, batch_size = batch_size_validacao)
    return dataloader_treino, dataloader_validacao

## Configuração do Accelerate

Para realizar o treinamento do modelo em diferentes ambientes, vamos utilizar o
`Accelerate`. Essa biblioteca consegue ajustar o código ao ambiente que está sendo utilizado e para isso é necessário fazer uma configuração. Vamos fazer isso usando o comando `accelerate config`.

In [ ]:
# !pip install -q accelerate

In [ ]:
!accelerate config

----------------------------------------------------------------------------------------------------In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)
0
This machine
----------------------------------------------------------------------------------------------------Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-GPU
    multi-NPU
    multi-MLU
    multi-MUSA
    TPU
0
No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon / Ascend NPU device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:NO
Do you want to use DeepSpeed? [yes/NO]: NO
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:all
Would you like to enable numa efficiency? (Currently

In [ ]:
!accelerate env


Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.33.0
- Platform: Linux-6.1.85+-x86_64-with-glibc2.35
- `accelerate` bash location: /usr/local/bin/accelerate
- Python version: 3.10.12
- Numpy version: 1.26.4
- PyTorch version (GPU?): 2.4.0+cu121 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- PyTorch MLU available: False
- PyTorch MUSA available: False
- System RAM: 12.67 GB
- GPU type: Tesla T4
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: NO
	- mixed_precision: bf16
	- use_cpu: False
	- debug: False
	- num_processes: 1
	- machine_rank: 0
	- num_machines: 1
	- gpu_ids: all
	- rdzv_backend: static
	- same_network: True
	- main_training_function: main
	- enable_cpu_affinity: False
	- downcast_bf16: no
	- tpu_use_cluster: False
	- tpu_use_sudo: False
	- tpu_env: []


In [ ]:
!accelerate test


Running:  accelerate-launch /usr/local/lib/python3.10/dist-packages/accelerate/test_utils/scripts/test_script.py
stderr: /usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
stderr:   self.scaler = torch.cuda.amp.GradScaler(**kwargs)
stdout: **Initialization**
stdout: Testing, testing. 1, 2, 3.
stdout: Distributed environment: NO
stdout: Num processes: 1
stdout: Process index: 0
stdout: Local process index: 0
stdout: Device: cuda
stdout: 
stdout: Mixed precision type: bf16
stdout: 
stdout: 
stdout: **Test process execution**
stdout: 
stdout: **Test split between processes as a list**
stdout: 
stdout: **Test split between processes as a dict**
stdout: 
stdout: **Test split between processes as a tensor**
stdout: 
stdout: **Test split between processes evenly**
stdout: 
stdout: **Test split between processes as a datasets.Dataset**
stdout: 
std

## Criando função do loop de treinamento

Agora que a configuração do Accelerate foi concluída, precisamos criar um loop de treinamento do modelo. Vamos realizar as configurações de hiperparâmetros, importar as bibliotecas necessárias e criar uma função para treinar e avaliar o modelo usando o `Accelerate`.

In [ ]:
hiperparametros = {
    'taxa_aprendizado' : 2e-5,
    'numero_epocas' : 2,
    'batch_size_treino' : 8,
    'batch_size_validacao': 32,
    'seed': 10
}

In [ ]:
# !pip install -q evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
import evaluate
import datasets
import transformers
import torch

from accelerate import Accelerator
from accelerate.utils import set_seed
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm.auto import tqdm

In [ ]:
acuracia = evaluate.load('accuracy')
metricas_agregadas = evaluate.combine(['f1', 'precision', 'recall'])

In [ ]:
def funcao_treinamento():

    # Configuração

    accelerator = Accelerator(mixed_precision = 'bf16')

    if accelerator.is_main_process:
        datasets.utils.logging.set_verbosity_warning()
        transformers.utils.logging.set_verbosity_info()
    else:
        datasets.utils.logging.set_verbosity_error()
        transformers.utils.logging.set_verbosity_error()

    dataloader_treino, dataloader_validacao = criar_dataloaders(
        batch_size_treino = hiperparametros['batch_size_treino'],
        batch_size_validacao = hiperparametros['batch_size_validacao']

    )

    set_seed(hiperparametros['seed'])

    checkpoint_modelo = 'Geotrend/bert-base-pt-cased'
    id2label = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
    label2id = {'entailment': 0, 'neutral': 1, 'contradiction': 2}

    modelo = AutoModelForSequenceClassification.from_pretrained(
        checkpoint_modelo,
        num_labels= 3,
        id2label = id2label,
        label2id = label2id
    )

    tokenizador = AutoTokenizer.from_pretrained(checkpoint_modelo)

    # Treinamento

    otimizador = AdamW(params = modelo.parameters(), lr = hiperparametros['taxa_aprendizado'])

    modelo, otimizador, dataloader_treino, dataloader_validacao = accelerator.prepare(
        modelo, otimizador, dataloader_treino, dataloader_validacao)

    numero_epocas = hiperparametros['numero_epocas']

    scheduler_taxa_aprendizado = get_linear_schedule_with_warmup(
        optimizer = otimizador,
        num_warmup_steps = 100,
        num_training_steps = len(dataloader_treino)*numero_epocas
    )

    barra_progresso = tqdm(range(numero_epocas*len(dataloader_treino)), disable = not accelerator.is_main_process)

    for epoca in range(numero_epocas):
        modelo.train()
        for passo, batch in enumerate(dataloader_treino):
            outputs = modelo(**batch)
            loss = outputs.loss
            accelerator.backward(loss)

            otimizador.step()
            scheduler_taxa_aprendizado.step()
            otimizador.zero_grad()
            barra_progresso.update(1)

    # Avaliação

        modelo.eval()

        lista_previsoes = []
        lista_labels = []

        for passo, batch in enumerate(dataloader_validacao):
            with torch.no_grad():
                outputs = modelo(**batch)
            previsoes = outputs.logits.argmax(dim = -1)

            lista_previsoes.append(accelerator.gather(previsoes))
            lista_labels.append(accelerator.gather(batch['labels']))

        lista_previsoes = torch.cat(lista_previsoes)[:len(dados_tokenizados['test'])]
        lista_labels = torch.cat(lista_labels)[:len(dados_tokenizados['test'])]

        metrica_avaliacao_acuracia = acuracia.compute(predictions = lista_previsoes, references = lista_labels)
        metricas_avaliacao_agregadas = metricas_agregadas.compute(predictions = lista_previsoes, references = lista_labels, average = 'macro')

        accelerator.print(f'Época: {epoca}\nAcurácia: {metrica_avaliacao_acuracia}\nMétricas agregadas: {metricas_avaliacao_agregadas}')

    modelo.push_to_hub('bert-pt-cased-zero-shot-anli')
    tokenizador.push_to_hub('bert-pt-cased-zero-shot-anli')

## Treinando o modelo

Para treinar o modelo usando a função dentro do notebook é bem simples, basta utilizar o comando `notebook_launcher` com a função de treinamento como parâmetro e uma tupla contendo o modelo e argumentos adicionais.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(funcao_treinamento)

Launching training on one GPU.


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-pt-cased/snapshots/793cd00d9242c56f3dccf438a75966f92035b487/config.json
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-pt-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "p

pytorch_model.bin:   0%|          | 0.00/421M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-pt-cased/snapshots/793cd00d9242c56f3dccf438a75966f92035b487/pytorch_model.bin
Some weights of the model checkpoint at Geotrend/bert-base-pt-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertFo

  0%|          | 0/5000 [00:00<?, ?it/s]

Época: 0
Acurácia: {'accuracy': 0.693}
Métricas agregadas: {'f1': 0.6784529475411644, 'precision': 0.6843718979754597, 'recall': 0.6764353655118551}
Época: 1
Acurácia: {'accuracy': 0.709}
Métricas agregadas: {'f1': 0.694514443346712, 'precision': 0.7014667275845382, 'recall': 0.6918713919442275}


Configuration saved in /tmp/tmp7qctesiz/config.json
Model weights saved in /tmp/tmp7qctesiz/model.safetensors
Uploading the following files to joaomiranda27/bert-pt-cased-zero-shot-anli: config.json,README.md,model.safetensors


model.safetensors:   0%|          | 0.00/421M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer config file saved in /tmp/tmp4nzz2a_m/tokenizer_config.json
Special tokens file saved in /tmp/tmp4nzz2a_m/special_tokens_map.json
Uploading the following files to joaomiranda27/bert-pt-cased-zero-shot-anli: special_tokens_map.json,tokenizer_config.json,tokenizer.json,vocab.txt,README.md


O aviso está nos dizendo que alguns pesos estão sendo eliminados e alguns estão sendo aleatoriamente inicializados. Além disso ele indica que precisamos treinar esse modelo em uma tarefa para ele poder fazer predições e inferência.

Esse aviso é normal já que estamos carregando um modelo BERT de base que não foi construído para fazer inferências e é justamente isso que vamos fazer. Vamos realizar a transfereência de aprendizado do modelo com os nossos dados.

Caso de uso em outro ambiente fora do colab:

```
!accelerate launch arquivo.py --configs
```

## Armazenando as informações do modelo no Hub

In [ ]:
evaluate.push_to_hub(
    model_id = 'joaomiranda27/bert-pt-cased-zero-shot-anli',
    metric_value = 0.709,
    metric_type = 'accuracy',
    metric_name = 'Acurácia',
    dataset_type = 'MoritzLaurer/multilingual-NLI-26lang-2mil7',
    dataset_name = 'Inferência de linguagem natrual',
    dataset_split = 'pt_anli',
    task_type = 'zero-shot-classification',
    task_name = 'Zero shot Classification'
)

CommitInfo(commit_url='https://huggingface.co/joaomiranda27/bert-pt-cased-zero-shot-anli/commit/42f36801cd4d15fba9eca251dd53d0497cf4ea37', commit_message='Update metadata with huggingface_hub', commit_description='', oid='42f36801cd4d15fba9eca251dd53d0497cf4ea37', pr_url=None, pr_revision=None, pr_num=None)

## Aplicando o modelo zero-shot

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizador = AutoTokenizer.from_pretrained('joaomiranda27/bert-pt-cased-zero-shot-anli')
modelo = AutoModelForSequenceClassification.from_pretrained('joaomiranda27/bert-pt-cased-zero-shot-anli')

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/162k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/554k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/tokenizer_config.json


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/config.json
Model config BertConfig {
  "_name_or_path": "joaomiranda27/bert-pt-cased-zero-shot-anli",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "neutral",
    "2": "contradiction"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 2,
    "entailment": 0,
    "neutral": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_h

model.safetensors:   0%|          | 0.00/421M [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--joaomiranda27--bert-pt-cased-zero-shot-anli/snapshots/42f36801cd4d15fba9eca251dd53d0497cf4ea37/model.safetensors
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at joaomiranda27/bert-pt-cased-zero-shot-anli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
premissa = 'O produto veio com defeito, quero um reembolso imediato'
hipotese = 'Reembolso'

In [ ]:
textos_tokenizados = tokenizador.encode(premissa, hipotese, return_tensors = 'pt', truncation_strategy = 'only_first')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2905: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


In [ ]:
textos_tokenizados

tensor([[   11,    60, 17709, 17981,   303, 23597, 15784,    25, 15894,   231,
           368,  1165,   480,  5627,  1179,  4669, 22380,   231,    12,  4784,
           480,  5627,  1179,    12]])

In [ ]:
logits = modelo(textos_tokenizados)[0]

In [ ]:
logits

tensor([[ 0.4994, -0.6799,  0.2566]], grad_fn=<AddmmBackward0>)

In [ ]:
entail_contradiction_logits = logits[:,[0,2]]

In [ ]:
entail_contradiction_logits

tensor([[0.4994, 0.2566]], grad_fn=<IndexBackward0>)

In [ ]:
probabilidades = entail_contradiction_logits.softmax(dim=1)
probabilidades

tensor([[0.5604, 0.4396]], grad_fn=<SoftmaxBackward0>)

## Aplicando o modelo com pipeline

In [ ]:
from transformers import pipeline

In [ ]:
classificador_zero_shot = pipeline('zero-shot-classification', model = modelo, tokenizer = tokenizador)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
texto = 'A qualidade do produto é excelente, muito bem feito e durável'
candidate_labels = ['preço', 'qualidade', 'entrega', 'plataforma de vendas']
classificador_zero_shot(texto, candidate_labels)

{'sequence': 'A qualidade do produto é excelente, muito bem feito e durável',
 'labels': ['qualidade', 'entrega', 'plataforma de vendas', 'preço'],
 'scores': [0.5190327763557434,
  0.23009581863880157,
  0.14837664365768433,
  0.1024947315454483]}

In [ ]:
texto = 'Fiquei impressionado com a eficiência da entrega, chegou muito rápido'
candidate_labels = ['preço', 'qualidade', 'entrega', 'plataforma de vendas']
classificador_zero_shot(texto, candidate_labels)

{'sequence': 'Fiquei impressionado com a eficiência da entrega, chegou muito rápido',
 'labels': ['entrega', 'qualidade', 'plataforma de vendas', 'preço'],
 'scores': [0.42811983823776245,
  0.27471256256103516,
  0.17750808596611023,
  0.11965947598218918]}

In [ ]:
texto = 'Pelo custo benefício, o preço é realmente muito bom'
candidate_labels = ['satisfação', 'insatisfação']
classificador_zero_shot(texto, candidate_labels)

{'sequence': 'Pelo custo benefício, o preço é realmente muito bom',
 'labels': ['satisfação', 'insatisfação'],
 'scores': [0.6672038435935974, 0.332796186208725]}

## Criando aplicação com gradio

In [ ]:
# !pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr

In [ ]:
classificador_zero_shot = pipeline('zero-shot-classification', model = modelo, tokenizer = tokenizador)
gr.Interface.from_pipeline(classificador_zero_shot).launch()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9c8e84e4a46f6c0d2a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Personalizando a aplicação do Gradio

In [ ]:
def classificacao_zero_shot(texto, tipo_classificacao, classificao_personalizada):
    classificador_zero_shot = pipeline('zero-shot-classification', model = modelo, tokenizer = tokenizador)
    if tipo_classificacao == 'Satisfação':
        classes = ['satisfação', 'insatisfação']
    elif tipo_classificacao == 'Motivo da mensagem':
        classes = ['preço', 'qualidade', 'entrega', 'plataforma de vendas']
    elif tipo_classificacao == 'Classificação personalizada':
        classes = classificao_personalizada.split(',')
    resultado = classificador_zero_shot(texto, classes)
    resultado = dict(zip(resultado['labels'], resultado['scores']))
    return resultado

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown(
        """
        # Aplicativo de classificação **Zero-Shot**!
        Escreva o texto e escolha as opções de classificação.
        """
    )

    texto = gr.Textbox(label= 'Escreva o texto')
    tipo_classificacao = gr.Dropdown(
        ['Satisfação', 'Motivo da mensagem', 'Classificação personalizada'],
        label = 'Opções de classificação',
        info = 'Escolha as opções que desja para realizar a classificação zero-shot'
    )
    classificacao_personalizada = gr.Textbox(label = 'Classificação personalizada',
                                             info = 'Escreva as labels candidatas separadas por vírgula')
    with gr.Row():
        with gr.Column(scale = 1):
            button = gr.Button(value = "Classificar o texto")
        with gr.Column(scale = 1):
            clear_button = gr.ClearButton(value = "Limpar")

    classificacao = gr.Label(label = 'Resuldado da classificação')

    button.click(fn = classificacao_zero_shot, inputs = [texto, tipo_classificacao, classificacao_personalizada], outputs = classificacao)
    clear_button.add([texto, tipo_classificacao, classificacao_personalizada,classificacao])

app.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6b7200c562921eab28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
